# Kalman Filter zur Zustandsschaetzung

_Philipp Rapp_

**Mechatronik 2022/2023**



In [ ]:
# Pakete
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

%matplotlib inline

## Erzeugung der Trajektorie des wahren Zustands

Da wir in der Simulation arbeiten, koennen wir die Trajektorie $x(t)$ des wahren (und eigentlich unbekannten) Zustands erzeugen.
Damit koennen wir die Funktion des Kalman Filters ueberpruefen.

Wir diskretisieren den Verlauf des Zustands mit der Schrittweite $\Delta T = 0.01$ (das entspricht 100 Hz) und speichern das Ergebnis in einem Dictionary.

Als Szenario nehmen wir an, dass das Flugzeug durch einen Punkt modelliert wird und eine Acht abfliegt.

Die (wahre) Position des Punktziels ist gegeben durch die Position $(p_x, p_y)$ mit
\begin{align}
    p_x &= \cos(\omega t) \\
    p_y &= \sin(2 \omega t)
\end{align}
mit $t \in [0,T]$.

Die Geschwindigkeit ergibt sich damit zu
\begin{align}
    v_x &= -\omega \sin(\omega t) \\
    v_y &= 2 \omega \cos(2 \omega t)
\end{align}

In [ ]:
T = 10
omega = 2*np.pi/T

true_state = {}
true_state['step_size'] = 0.01
# 0 bis 10 Sekunden
true_state['t'] = np.arange(0, 10.0, true_state['step_size'])
true_state['pos_x'] = np.cos(omega*true_state['t'])
true_state['pos_y'] = np.sin(2.0*omega*true_state['t'])
true_state['x'] = np.column_stack((true_state['pos_x'], true_state['pos_y']))

In [ ]:
plt.plot(true_state['pos_x'], true_state['pos_y'])
plt.grid(True)
plt.xlabel('pos_x')
plt.ylabel('pos_y')
plt.show()

## Hinzufuegen von Messrauschen

Als naechstes fuegen wir Messrauschen hinzu. Wir nehmen an, dass das Messrauschen mittelwertfrei, normalverteilt und weiss ist. Weisses Rauschen bedeutet, dass es zeitlich unkorreliert ist.



In [ ]:
R = 0.02**2 * np.eye(2)
print(f'Kovarianzmatrix R des Messrauschens:\n{R}')

measurement_noise = { 'mean': np.array([0.0, 0.0]), 'cov': R }
measurement_noise_distribution = multivariate_normal(mean=measurement_noise['mean'], cov=measurement_noise['cov'])

## Erzeugung des Messsignals

Das gemessene Signal $y(t)$ besteht aus der Position, die zusaetzlich durch Rauschen ueberlagert ist.

Wir nehmen die gleiche zeitliche Diskretisierung an wie fuer den wahren Zustand.

In [ ]:
N = len(true_state['t'])

measurement_signal = {}
measurement_signal['t'] = true_state['t']
measurement_signal['y'] = true_state['x'] + measurement_noise_distribution.rvs(N)

In [ ]:
plt.plot(true_state['x'][:,0], true_state['x'][:,1], '--', color=[0.5,0.5,0.5], label='Wahrer Zustand')
plt.scatter(measurement_signal['y'][:,0], measurement_signal['y'][:,1], s=1.2, label='Messwerte')
plt.grid(True)
plt.legend()
plt.show()

## Auslegung des Kalman Filters

### Streckenmodell
Im Kontext der Zustandsschaetzung eines bewegten Ziels wird
das Streckenmodell oft auch _Bewegungsmodell_ genannt,
da es die Bewegung des Ziels modelliert.

Wir gehen von einem Punktziel aus und formulieren den Schwerpunktsatz
\begin{align}
    m \ddot{p}_x &= F_x \\
    m \ddot{p}_y &= F_y
\end{align}

Da wir die Kraefte nicht kennen, setzen wir sie zu Null.
Den Fehler, den wir dadurch erhalten, modellieren wir durch das sogenannte _Prozessrauschen_.
Dieses wird dazu genutzt, Unsicherheiten im Streckenmodell zu modellieren.

Unter Annahme einer konstanten Masse $m$ erhalten wir
\begin{align}
    \dot{v}_x &= 0 \\
    \dot{v}_y &= 0
\end{align}

Dieses Modell ist in der Literatur unter dem Namen _Constant Velocity_ Modell bekannt.

### Messmodell

### Zustandsraumdarstellung
Wir koennen das Streckenmodell in Form der linearen ZRD darstellen.

### Zeitliche Diskretisierung

